# Building an unstructured mesh

<div class="alert alert-block alert-info"> In this example, we use the ESRI Grid for Australia available from Geoscience Australia product catalogue (https://ecat.ga.gov.au/geonetwork/). You could download it when searching for Australian Bathymetry and Topography Grid, June 2009.  

We also provide in data folder a low resolution GeoTIFF that can also be used for this tutorial (AUS_LR.tiff). 

We will first _reproject the dataset_ in UTM coordinates, then we will use _shapefiles and countours_ to clipped on region of interested and then we will use


</div>



## Notebook contents

   - [Converting from lon/lat to metres](#Converting-from-lon/lat-to-metres)
   - [Clipped elevation grid](#Clipped-elevation-grid)
   - [X & Y axes](#X-&-Y-axes)
   - [Define contour lines](#Define-contour-lines)
   - [Unstructured elevation grid](#Unstructured-elevation-grid)
   - [Visualisation](#Visualisation)


In [ ]:
import meshio
import numpy as np
from scipy.interpolate import RectBivariateSpline
import jigsawpy
import os
from collections import OrderedDict

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable

label_size = 8
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

# Converting from lon/lat to metres

To reproject the grid from lon/lat coordinates to UTM (metres), two main libraries are available within the Docker image:

+ `pygeotools` -- https://github.com/dshean/pygeotools
+ `rasterio` -- https://github.com/mapbox/rasterio

First, we specify our DEM filename:

In [ ]:
filename = 'input_data/AUS_LR.tiff'

Below, we show how this can be done using rasterio. First we load the required libraries and then define the requested projection (here we used EPSG reference for the region EPSG:28355).

In [ ]:
import rasterio
from rasterio import crs
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio import drivers

# Reproject to EPSG zone
dst_crs = {'init': 'EPSG:28355'}

# Requested reprojected dataset resolution (metres)
utmRes = 10000.

We then use the following cell to make the projection and get the interpolated elevation points at the requested resolution (elev -- a numpy masked array)

In [ ]:
with rasterio.open(filename) as src:
    array = src.read(1)
    print(array.shape, array.dtype)
    
    plt.imshow(array, cmap='pink')

In [ ]:
with rasterio.open(filename) as src:

    profile = src.profile
    print(profile)
    if src.nodata is None:
        nodata = -100000
    else:
        nodata = src.nodata

    if src.crs is None:
        src_crs = src_crs
    else:
        src_crs = src.crs

    # Calculate the ideal dimensions and transformation in the new crs
    dst_affine, dst_width, dst_height = calculate_default_transform(
        src_crs, dst_crs, src.width, src.height, *src.bounds, resolution=utmRes)

    # update the relevant parts of the profile
    profile.update({
        'crs': dst_crs,
        'transform': dst_affine,
        'width': dst_width,
        'height': dst_height,
    })

    # Reproject and write each band
    src_array = src.read()
    dst_array = np.empty((int(dst_height), int(dst_width)), dtype='float32')

    reproject(
            # Source parameters
            source=src_array,
            src_crs=src_crs,
            src_transform=src.transform,
            src_nodata=nodata,

            # Destination paramaters
            destination=dst_array,
            dst_transform=dst_affine,
            dst_crs=dst_crs,
            dst_nodata=nodata,

            # Configuration
            resampling=Resampling.nearest,
            num_threads=2)

    elev = np.ma.masked_where(dst_array == nodata, dst_array)

We can look at the metadata associated with the new GeoTIFF file using for example:

In [ ]:
profile

# Clipped elevation grid

We can visualise the new elevation array using the following function:

In [ ]:
def plotElevation( data, cmin, cmax, colormap):
    '''
    data: dataset to plot
    zmin,zmax: extent of the colormap
    colormap: to use    
    '''
    
    # Figure size is defined here
    fig = plt.figure(1, figsize=(8,8))
    
    ax = plt.gca()
    im = ax.imshow(data, interpolation='nearest', cmap=colormap,
                     vmin=cmin, vmax=cmax)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    cbar = plt.colorbar(im,cax=cax)
    plt.tight_layout()

    plt.show()
    plt.close(fig)

In [ ]:
vis_max_elevation =  2000
vis_min_elevation = -1000

topocmap = 'pink'
plotElevation( elev, vis_min_elevation, vis_max_elevation, topocmap)

In [ ]:
dem = elev

# X & Y axes

To keep your coordinates system for post-processing and to potentially reproject the outputs from the landscape evolution model in another geospatial system we needs to specify the X and Y axes.
We do it like this

In [ ]:
xMin = dst_affine[2]
xMax = dst_affine[2] + abs(dst_affine[0])*dst_width

yMin = dst_affine[5] - abs(dst_affine[4])*dst_height
yMax = dst_affine[5]

print("Initial DEM:\n")

print("Lower left coordinates       Xmin: {}, Ymin: {}".format(xMin,yMin))
print("Upper right coordinates      Xmax:  {}, Ymax: {}".format(xMax,yMax))

We can now create the X and Y coordinates, at this point we can choose to decrease the resolution if needed by using the step parameter (integer)

In [ ]:
step = 1
spacing = utmRes*step

Z = dem[::step,::step]

nx = Z.shape[1]
ny = Z.shape[0]

minX, maxX = xMin, xMin+spacing*nx
minY, maxY = yMin, yMin+spacing*ny

xcoords = np.arange(minX, maxX, spacing)
ycoords = np.arange(minY, maxY, spacing)

X, Y = np.meshgrid(xcoords, ycoords)

coords = np.vstack([X.ravel(), Y.ravel()])

print("Clipped DEM:\n")

print("Resolution (m)            res: {}".format(spacing))
print("Number of points         nbpt: {}".format(coords.shape[0]))
print("Elevation map shape        nx: {}, ny: {}\n".format(nx,ny))

print("Lower left coordinates   Xmin: {}, Ymin: {}".format(minX,minY))
print("Upper right coordinates  Xmax: {}, Ymax: {}".format(maxX,maxY))

# Define contour lines

From the projected digital elevation, we will extract contour lines at given depth and use these lines to define the extent of our simulation region and its resolution. 

First we define the `extractContours` function that returns the list of countour lines

In [ ]:
def extractContours( X, Y, Z, cmin, cmax, colormap, ctrlist):
    '''
    coords: coordinate points (X,Y,X)
    cmin,cmax: extent of the colormap
    colormap: color scale to use
    ctrlist: list of contours to extract
    '''
    # Figure size is defined here
    fig = plt.figure(1, figsize=(8,8))
    ctrs = []
    for k in range(len(ctrlist)):
        print(k, len(ctrlist), ctrlist[k], type(ctrlist[k]))
        ctrs.append(plt.contour(X, Y, 
                    np.flipud(Z), [ctrlist[k]]))
    ax = plt.gca()
    im = ax.imshow(Z, interpolation='nearest', cmap=colormap,
                     vmin=cmin, vmax=cmax,extent=[minX, maxX,minY, maxY])
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    cbar = plt.colorbar(im,cax=cax)
    plt.tight_layout()

    plt.show()
    plt.close(fig)

    return ctrs

Then we specify a list of contour line depths `ctrList` that needs to be defined in **ascending order** (this is important for what follows)

In [ ]:
depth_contour = -12.


ctrList = [depth_contour,]
# Now we extract the contours lines using the previous function
if sorted(ctrList) == ctrList:
    ctrs = extractContours(X, Y, Z, vis_min_elevation, vis_max_elevation, topocmap, ctrList)
else:
    print("ERROR:")
    print("The list of contour positions needs to be specify in ascending order!")
    print("Redefine the ctrList variable")

In [ ]:
print(ctrs)

We can see from the figure above that we have several contour lines for any single depth. We will only use the **longest lines for each depth** to define our simulation domain.

To do so we will define two functions:
+ `distancePts`: that will be used to get the euclidian distance between 2 points
+ `getLongestContoursXY`: that extract the longest lines and resample it based on a characteristic length `lcar`

In [ ]:
def distancePts(p1,p2):
    '''
    Compute the euclidian distance between 2 points (p1, p2)
    
    '''
    return (p1[1]-p2[1])**2+(p1[0]-p2[0])**2 

In [ ]:
def getLongestContoursXY(ctrs, lcar):
    '''
    1- Extract from the list of contour points the longest path
    2- Using a characteristic length (lcar) resample the path 
    3- Return the XY coordinates of the longest paths
    '''

    ctrPoints = []
    # Loop through the contour lines 
    for ct in range(len(ctrs)):
        cpath = []
        k = 0
        maxpts = 0
        pathID = 0
        
        # For each contour extract the longest path
        for collection in ctrs[ct].collections:
            for path in collection.get_paths():
                if len(path)>4:
                    cpath.append(np.asarray(path.to_polygons()[0]))
                    # Storing longest path
                    if cpath[-1].shape[0] > maxpts:
                        maxpts =  cpath[-1].shape[0]
                        pathID = k
                    k += 1

        # Find longest path XY coordinates 
        Cpts = cpath[pathID]
        x = Cpts[:,0]
        y = Cpts[:,1]
        tmp = OrderedDict()
        for pt in zip(x,y):
            tmp.setdefault(pt[:1], pt)   
        ctrPts = np.asarray(list(tmp.values()))
        # Resample the path to the requested characteristic length
        ki = 0
        tmpPts = []
        cumdist = 0.
        tmpPts.append(ctrPts[0,:])
        for k in range(1,ctrPts.shape[0]):
            cumdist = distancePts(ctrPts[ki,:2], ctrPts[k,:2])
            if(cumdist >= lcar):
                tmpPts.append(ctrPts[k,:])
                ki = k
        ctrPoints.append(np.asarray(tmpPts))

    return ctrPoints

The `getLongestContoursXY` function will return the longest line resampled points coordinates for each contour depths defined in `ctrList`

In [ ]:
ctrPoints = getLongestContoursXY(ctrs,1.)

Let's plot the picked contour lines...

In [ ]:
def plotContours( X, Y, Z, cmin, cmax, colormap, ctrPts):
    '''
    coords: coordinate points (X,Y)
    zmin,zmax: extent of the colormap
    colormap: to use  
    ctrPts: coordinates of contour lines
    '''
    # Figure size is defined here
    fig = plt.figure(1, figsize=(8,8))
    ctrs = []
    for k in range(len(ctrPts)):
        plt.scatter(ctrPts[k][:,0], ctrPts[k][:,1], s=0.3, c='k')
    ax = plt.gca()
    im = ax.imshow(Z, interpolation='nearest', cmap=colormap,
                     vmin=cmin, vmax=cmax,extent=[minX, maxX,minY, maxY])
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    cbar = plt.colorbar(im,cax=cax)
    plt.tight_layout()

    plt.show()
    plt.close(fig)

    return 

In [ ]:
plotContours(X, Y, Z, vis_min_elevation, vis_max_elevation, topocmap, ctrPoints)

# Perform the meshing

We use the [JigsawPy](https://github.com/dengwirda/jigsaw-python) tool to perform triangular meshing, with mesh refinement based on an arbitrary function - in this case, the mesh is refined in areas of high slope.

In [ ]:
# choose a folder to store all the output data
dst_path = "output_data"

# setup jigsaw mesh storage
opts = jigsawpy.jigsaw_jig_t()
geom = jigsawpy.jigsaw_msh_t()
mesh = jigsawpy.jigsaw_msh_t()
# refinement function mesh
hfun = jigsawpy.jigsaw_msh_t()

opts.geom_file = os.path.join(dst_path, "pslg.msh")
opts.jcfg_file = os.path.join(dst_path, "pslg.jig")
opts.mesh_file = os.path.join(dst_path, "mesh.msh")
opts.hfun_file = os.path.join(dst_path, "spac.msh")

# The output name of the mesh we'll use for gLEC
output_vtk = os.path.join(dst_path, "AUS_LR.vtk")

## Setup the data

We take the contour points found in the previous step, and create a closed polygon out of them.

We do this by first adding all the contour points as vertexes, and then we link each vertex with edges. Notice we also link the first and last vertexes with an edge too, so the loop is closed.

In [ ]:
#######################################
# Setup contour points as a big polygon
geom.mshID = "euclidean-mesh"
geom.ndims = +2

# For each point in the contour, add it as a vertex
arr = []
for i in range(ctrPoints[0].shape[0]):
    arr.append(([ctrPoints[0][i][0], ctrPoints[0][i][1]], 0))
    
arr = np.array(arr, dtype=geom.VERT2_t)
geom.vert2 = arr

# Now link each vertex with a line, remembering to link the first
# and last points too.
lineLoop=[]
for i in range(ctrPoints[0].shape[0]):
    if i < ctrPoints[0].shape[0]-1:
        lineLoop.append(([i, i+1], 0))
    else:
        lineLoop.append(([i, 0], 0))
edges = np.array(lineLoop, dtype=geom.EDGE2_t)
geom.edge2 = edges

jigsawpy.savemsh(opts.geom_file, geom)
#
#######################################

## Mesh refinement

We define the arbitrary function in the `hfun` mesh. This is a euclidean grid, and the value at each point will determine how well refined the final triangular mesh should be in that area.

In [ ]:
hfun.mshID = "euclidean-grid"
hfun.ndims = +2

hfun.xgrid = xcoords
hfun.ygrid = ycoords

# The approximate min and max size for the final triangular mesh cells
hmin = 10000.
hmax = 200000.

### Choosing where to refine

Since gLEC is finding the connectivity of different elevations, we need to get as much detail as possible in areas where elevation is changing. Therefore, the gradient of elevation, or the slope, is a good proxy for choosing where to have a higher resolution.

Here we define a grid of the gradient of elevation, with some processing steps to smooth it out and clamp the variance.

In [ ]:
# Get a copy of the data into a jigsaw compatible format
out = np.flipud(Z).astype(dtype=jigsawpy.jigsaw_msh_t.REALS_t, copy=True)
sea_mask = out < depth_contour
land_mask = np.logical_not(sea_mask)

# Take the magnitude of the gradient of the topo, 
grad = np.linalg.norm(np.gradient(out), axis=0)
# Set the gradient to be 0 in areas we don't care about
grad[sea_mask] = 0.

# Take the squareroot (or something similar) to reduce the variance of the gradient
# Varying the power will change how 'focused' the refinement is in areas of high
# gradient.
scaled = np.power(grad, 0.15)
smax = np.max(scaled[land_mask])

# While this is not a normal distribution, we can use these stats
# to help focus the refinement.
land_std  = np.std(scaled[land_mask])
land_mean = np.mean(scaled[land_mask])

# Use the stats to clamp the hfun, so we get better meshing.
scaled[scaled < land_mean - land_std] = land_mean - land_std
scaled[scaled > land_mean + land_std] = land_mean + land_std

# Flip the sign of the gradient field, so areas with high gradient have small numbers
# (corresponding with small mesh size)
scaled = smax - scaled

# Now scale the gradient to fit between the min and max size for the mesh
scaled = ((scaled - np.min(scaled)) / (np.max(scaled) - np.min(scaled))) * (hmax - hmin) + hmin

# Now transfer the finalised function into the hfun mesh
hfun.value = scaled

jigsawpy.savemsh(opts.hfun_file, hfun)

# Apply the refinement
jigsawpy.cmd.marche(opts, hfun)

#
#######################################

#### Refinement output

We can visualise the hfun mesh, and see which areas will have higher resolution (smaller values), and which areas will have lower resolution (higher values).

In [ ]:
plt.imshow(np.flipud(hfun.value), vmin=hmin, vmax=hmax, cmap='viridis')
plt.colorbar()

#### Outputting the mesh

Now we can tell jigsaw to generate the final mesh, populate its vertexes with elevation data, and write it to file.

In [ ]:
opts.hfun_scal = "absolute"
opts.hfun_hmin = hmin
opts.hfun_hmax = hmax

opts.mesh_dims = +2                 # 2-dim. simplexes

# If you need the contour line to be well preserved
# enable geom_feat, and use geom_eta1 to control how fine
# it should be
#opts.geom_feat = True               # do sharp feature
#opts.geom_eta1 = 45.
opts.mesh_top1 = True               # preserve 1-topo.

opts.mesh_eps1 = 1.0                # relax edge error

opts.optm_qlim = +9.5E-01           # tighter opt. tol
opts.optm_iter = +32
opts.optm_qtol = +1.0E-05

jigsawpy.cmd.jigsaw(opts, mesh)

# Populate the Jigsaw mesh vertexes with the topographic data
zfun = RectBivariateSpline(xcoords,ycoords,np.flipud(Z).T)
mesh.value = zfun(mesh.point['coord'][:,0], mesh.point['coord'][:,1], grid=False)


# Save the mesh. We can now use this for gLEC, or view it in Paraview
outmesh = meshio.Mesh(mesh.point['coord'], {'triangle': mesh.tria3['index']}, {'Z':mesh.value})
meshio.write(output_vtk, outmesh)

# Quick visualation

We can have a simple preview of the mesh by using Matplotlib, however, the outputs are best viewed in a tool like Paraview.

In [ ]:
import matplotlib.tri as mtri

triang = mtri.Triangulation(mesh.point['coord'][:,0], mesh.point['coord'][:,1], mesh.tria3['index'])
fig, ax = plt.subplots()
ax.triplot(triang)
ax.set_aspect('equal')
